In [8]:
import math
import random
import numpy as np

In [217]:
# sta_amount: Total contend stations number
# RA_RU: RA_RU number reserved for UORA
# Lmax: Retransmit limit
# TXOP: Transaction Opportunity (time length in ms)
# ocw_min: minimum OCW
# ocw_max: maximum OCW
# CR: Capture Ratio (CR=infinite = no capture, CR=0 = always capture)

def simulation(sta_amount, RA_RU, Lmax, TXOP, ocw_min, ocw_max, CR, step, gamma):
    total_power = 0
    counter_delay_slot=0
    success_sta=0
    # setting ocw for each transmission [ocw_min, 1st_retry_ocw...ocw_max]
    ocw = [0 for _ in range(0, Lmax)]
    ocw[0] = ocw_min
    for n in range (1, Lmax):
        temp_ocw = ocw[n-1] * 2 + 1
        if temp_ocw <= ocw_max:
            ocw[n] = temp_ocw
        else:
            ocw[n] = ocw[n-1]
    # Imax: Reserved slot for transmission (total slots in worst case)
    Imax = 0
    for n in range(0, Lmax):
        Imax += math.ceil(ocw[n] / RA_RU)

    # ------------------------------------ STA setting ------------------------------------------
    # set each STA's OBO counter, obo_counter[STA1_obo_counter, STA2_obo_counter...]
    obo_counter = [random.randint(0, ocw_min) for _ in range(0, sta_amount)]
    # set STA's transmit attempt time, retrans_counter[STA1_obo_counter, STA2_obo_counter...]
    retrans_counter = [0 for _ in range(0, sta_amount)]
    # STA's distance from AP (From 0~1)
    sta_distance = [random.uniform(0, 1) for _ in range(0, sta_amount)]
    # record STAs choosen RA-RU, sta_on_R[slot_number][STA_1, STA_26...]
    sta_on_R = [[[] for _ in range(0,RA_RU)] for _ in range(0,Imax)]
    # Accumulated station transmit power usage
    sta_power_usage = [0 for _ in range(0, sta_amount)]
    
    capture_power_ratio = 10 ** (CR / 10)

    # 第一步: **$M_i$個STA隨機選擇 $R$ 個RA-RU，剛好 $x$ 個STA選到第一個RA-RU (碰撞)**
    first_RU_have_2_STA_count = [0 for _ in range(0,Imax)]
    # 第二步：**在碰撞的x個STAs裡有y個第n次傳送STA** (設定n=2) [n][i]
    collision_STA_have_0_nth_count = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    collision_STA_have_1_nth_count = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    collision_STA_have_2_nth_count = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    # 第三步：**在x個STA碰撞下，裡面有y個第n次傳送STA，第n次傳送STA被捕獲**
    have_1_nth_capture_count = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    have_2_nth_capture_count = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    
    result_divider_2 = [0 for _ in range(0,Imax)] 
    # have_1_nth
    result_divider_3_1 = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    # have_2_nth
    result_divider_3_2 = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
    # No.i slot    
    for i in range(0, Imax):
        # STA's Transmit power [1P, 2P, 4P, 8P]
#         sta_power = [2 ** random.randint(0, 3) for _ in range(0, sta_amount)]
        sta_power = [2 ** (x if x <= (step - 1) else (step - 1)) for x in retrans_counter]

        # No.x STA
        for x in range (0,sta_amount):
            # decrease OBO counter by RA-RU number
            obo_counter[x] -= RA_RU
            # if counter <=0 select R
            if (obo_counter[x] <= 0 and retrans_counter[x] < Lmax):
                selected_R_index = random.randint(0, RA_RU-1)
                sta_on_R[i][selected_R_index].append(x)
                
                # record power usage
                sta_power_usage[x] += sta_power[x]
#                 sta_power_usage[x] += (2 ** retrans_counter[x])
        # check the balls in R
        for r in range(0, len(sta_on_R[i])):
            # Record power (no power control so always 1)
            for sta_index in sta_on_R[i][r]:
                total_power += sta_power[sta_index]
            # success STA
            if (len(sta_on_R[i][r]) == 1):
                success_sta_index = sta_on_R[i][r][0]
                # mark 99999 = won't trasnmit again
                obo_counter[success_sta_index] = 99999
                # record access delay slot number
                counter_delay_slot += (i+1)
                success_sta += 1
#                 total_power += sta_power_usage[success_sta_index]
            # 2 STA on the same R -> capture process
            elif (len(sta_on_R[i][r]) == 2):
                # divide result 1
                if (r == 0):
                    first_RU_have_2_STA_count[i] += 1;
                
                success_sta_index_1 = sta_on_R[i][r][0]
                success_sta_index_2 = sta_on_R[i][r][1]
                receive_power_1 = sta_power[success_sta_index_1] / (sta_distance[success_sta_index_1] ** gamma)
                receive_power_2 = sta_power[success_sta_index_2] / (sta_distance[success_sta_index_2] ** gamma)
                
                # divide result 2
#                 if (first_RU_have_2_STA_count[i]):
                result_divider_2[i] += 1
                # 0 n_th 
                for other_n in range(Lmax):
                        if other_n != retrans_counter[success_sta_index_1] and other_n != retrans_counter[success_sta_index_2]:
                            collision_STA_have_0_nth_count[other_n][i] += 1
                # 2 n-th
                if (retrans_counter[success_sta_index_1] == retrans_counter[success_sta_index_2]):
                    result_divider_3_2[retrans_counter[success_sta_index_1]][i] += 1
                    collision_STA_have_2_nth_count[retrans_counter[success_sta_index_1]][i] += 1
                # 1 n-th
                else:
                    result_divider_3_1[retrans_counter[success_sta_index_1]][i] += 1
                    result_divider_3_1[retrans_counter[success_sta_index_2]][i] += 1
                    collision_STA_have_1_nth_count[retrans_counter[success_sta_index_1]][i] += 1
                    collision_STA_have_1_nth_count[retrans_counter[success_sta_index_2]][i] += 1
#                 if collision_STA_have_1_nth_count[retrans_counter[success_sta_index_1]][i] > 1:
#                 print(f'n: {n}, i:{i}')
#                 print(f'retrans_counter[success_sta_index_1]: {retrans_counter[success_sta_index_1]}')
#                 print(f'retrans_counter[success_sta_index_2]: {retrans_counter[success_sta_index_2]}')
#                 print(f'{collision_STA_have_1_nth_count[retrans_counter[success_sta_index_1]][i]}')
                
                # capture success_sta_index_1
                if (receive_power_1 / receive_power_2) > capture_power_ratio:
                    # divide result 3
                    
                    if (retrans_counter[success_sta_index_1] != retrans_counter[success_sta_index_2]):
                        have_1_nth_capture_count[retrans_counter[success_sta_index_1]][i] += 1;
#                     if (collision_STA_have_2_nth_count[retrans_counter[success_sta_index_1]][i]):
                    else:
                        # 2 n-th STA so just add 1
                        have_2_nth_capture_count[retrans_counter[success_sta_index_1]][i] += 1;
                        
                    # Capture sta 1
                    obo_counter[success_sta_index_1] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
#                     total_power += sta_power_usage[success_sta_index_1]
                    
                    # fail STAs 2
                    retrans_counter[success_sta_index_2] += 1
                    if (retrans_counter[success_sta_index_2] < Lmax):
                        obo_counter[success_sta_index_2] = random.randint(0, ocw[retrans_counter[success_sta_index_2]])
                elif (receive_power_1 / receive_power_2) < (1 / capture_power_ratio):
                    # divide result 3
                    if (retrans_counter[success_sta_index_1] != retrans_counter[success_sta_index_2]):
                        have_1_nth_capture_count[retrans_counter[success_sta_index_2]][i] += 1;
#                     if (collision_STA_have_2_nth_count[retrans_counter[success_sta_index_2]][i]):
                    else:
                        # 2 n-th STA so just add 1
                        have_2_nth_capture_count[retrans_counter[success_sta_index_2]][i] += 1;
                    
                    # Capture sta 2
                    obo_counter[success_sta_index_2] = 99999
                    counter_delay_slot += (i+1)
                    success_sta += 1
                    
                    # fail STA 1
                    retrans_counter[success_sta_index_1] += 1
                    if (retrans_counter[success_sta_index_1] < Lmax):
                        obo_counter[success_sta_index_1] = random.randint(0, ocw[retrans_counter[success_sta_index_1]])
                else:
                    retrans_counter[success_sta_index_1] += 1
                    if (retrans_counter[success_sta_index_1] < Lmax):
                        obo_counter[success_sta_index_1] = random.randint(0, ocw[retrans_counter[success_sta_index_1]])
                    retrans_counter[success_sta_index_2] += 1
                    if (retrans_counter[success_sta_index_2] < Lmax):
                        obo_counter[success_sta_index_2] = random.randint(0, ocw[retrans_counter[success_sta_index_2]])
                
            else:
                for fail_idx in sta_on_R[i][r]:
                    retrans_counter[fail_idx] += 1
                    if (retrans_counter[fail_idx] < Lmax):
                        obo_counter[fail_idx] = random.randint(0, ocw[retrans_counter[fail_idx]])

#     for a in collision_STA_have_1_nth_count:
#     collision_STA_have_1_nth_count = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_2)] for row in collision_STA_have_1_nth_count]
#     collision_STA_have_2_nth_count = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_2)] for row in collision_STA_have_2_nth_count]
#     collision_STA_have_1_nth_count = [[x / result_divider_2 for x in row] for row in collision_STA_have_1_nth_count]
    return first_RU_have_2_STA_count, collision_STA_have_0_nth_count, collision_STA_have_1_nth_count, collision_STA_have_2_nth_count, have_1_nth_capture_count, have_2_nth_capture_count, result_divider_2, result_divider_3_1, result_divider_3_2

In [221]:
sta_amount = 100
Imax = 0
Lmax = 5
RA_RU = 9
ocw_min = 7
ocw_max = 31
ocw = [0 for _ in range(0, Lmax)]
ocw[0] = ocw_min
for n in range (1, Lmax):
    temp_ocw = ocw[n-1] * 2 + 1
    if temp_ocw <= ocw_max:
        ocw[n] = temp_ocw
    else:
        ocw[n] = ocw[n-1]
# Imax: Reserved slot for transmission (total slots in worst case)
Imax = 0
for n in range(0, Lmax):
    Imax += math.ceil(ocw[n] / RA_RU)
    
    
# 第一步: **$M_i$個STA隨機選擇 $R$ 個RA-RU，剛好 $x$ 個STA選到第一個RA-RU (碰撞)**
first_RU_have_2_STA_total = [0 for _ in range(0,Imax)]
# 第二步：**在碰撞的x個STAs裡有y個第n次傳送STA** (設定n=2) [n][i]
collision_STA_have_0_nth_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
collision_STA_have_1_nth_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
collision_STA_have_2_nth_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
# 第三步：**在x個STA碰撞下，裡面有y個第n次傳送STA，第n次傳送STA被捕獲**
have_1_nth_capture_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
have_2_nth_capture_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]

result_divider_2_total = [0 for _ in range(0,Imax)] 
result_divider_3_1_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]
result_divider_3_2_total = [[0 for _ in range(0,Imax)] for _ in range(0,Lmax)]

sample = 1000
for i in range (sample):
    first_RU_have_2_STA_count, collision_STA_have_0_nth_count, collision_STA_have_1_nth_count, collision_STA_have_2_nth_count, have_1_nth_capture_count, have_2_nth_capture_count, result_divider_2, result_divider_3_1, result_divider_3_2 = simulation(sta_amount=sta_amount, RA_RU=RA_RU, Lmax=Lmax, TXOP=5.673, ocw_min=ocw_min, ocw_max=ocw_max, CR=1.5, step=4, gamma=2)
#     print(f'first_RU_have_2_STA_count: {first_RU_have_2_STA_count}')
#     print(f'collision_STA_have_1_nth_count: {collision_STA_have_1_nth_count}')
#     print(f'collision_STA_have_2_nth_count: {collision_STA_have_2_nth_count}')
#     print(f'have_1_nth_capture_count: {have_1_nth_capture_count}')
#     print(f'have_2_nth_capture_count: {have_2_nth_capture_count}')
    first_RU_have_2_STA_total = [x + y for x, y in zip(first_RU_have_2_STA_total, first_RU_have_2_STA_count)]
    collision_STA_have_0_nth_total = [[collision_STA_have_0_nth_total[i][j] + collision_STA_have_0_nth_count[i][j] 
                      for j in range(len(collision_STA_have_0_nth_count[0]))] 
                     for i in range(len(collision_STA_have_0_nth_count))]
    
    collision_STA_have_1_nth_total = [[collision_STA_have_1_nth_total[i][j] + collision_STA_have_1_nth_count[i][j] 
                      for j in range(len(collision_STA_have_1_nth_count[0]))] 
                     for i in range(len(collision_STA_have_1_nth_count))]
    collision_STA_have_2_nth_total = [[collision_STA_have_2_nth_total[i][j] + collision_STA_have_2_nth_count[i][j] 
                      for j in range(len(collision_STA_have_2_nth_count[0]))] 
                     for i in range(len(collision_STA_have_2_nth_count))]
    
    have_1_nth_capture_total = [[have_1_nth_capture_total[i][j] + have_1_nth_capture_count[i][j] 
                      for j in range(len(have_1_nth_capture_total[0]))] 
                     for i in range(len(have_1_nth_capture_total))]
    have_2_nth_capture_total = [[have_2_nth_capture_total[i][j] + have_2_nth_capture_count[i][j] 
                      for j in range(len(have_2_nth_capture_total[0]))] 
                     for i in range(len(have_2_nth_capture_total))]
    
    result_divider_2_total = [result_divider_2_total[i] + result_divider_2[i] 
                      for i in range(len(result_divider_2_total))]
    result_divider_3_1_total = [[result_divider_3_1_total[i][j] + result_divider_3_1[i][j] 
                      for j in range(len(result_divider_3_1_total[0]))] 
                     for i in range(len(result_divider_3_1_total))]
    result_divider_3_2_total = [[result_divider_3_2_total[i][j] + result_divider_3_2[i][j] 
                      for j in range(len(result_divider_3_2_total[0]))] 
                     for i in range(len(result_divider_3_2_total))]
#     result_divider_3_1_total = [result_divider_3_1_total[i] + result_divider_3_1[i] 
#                       for i in range(len(result_divider_3_1_total))]
#     result_divider_3_2_total = [result_divider_3_2_total[i] + result_divider_3_2[i] 
#                       for i in range(len(result_divider_3_2_total))] 
                     
#     print(f'collision_STA_have_1_nth_total: {collision_STA_have_1_nth_total}')

first_RU_have_2_STA = [x / sample for x in first_RU_have_2_STA_total]
collision_STA_have_0_nth = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_2_total)] for row in collision_STA_have_0_nth_total]
collision_STA_have_1_nth = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_2_total)] for row in collision_STA_have_1_nth_total]
collision_STA_have_2_nth = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_2_total)] for row in collision_STA_have_2_nth_total]

have_1_nth_capture = [[num / denom if denom != 0 else 0 for num, denom in zip(row1, row2)] 
                  for row1, row2 in zip(have_1_nth_capture_total, result_divider_3_1_total)]
have_2_nth_capture = [[num / denom if denom != 0 else 0 for num, denom in zip(row1, row2)] 
                  for row1, row2 in zip(have_2_nth_capture_total, result_divider_3_2_total)]
# have_1_nth_capture = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_3_1_total)] for row in have_1_nth_capture_total]
# have_2_nth_capture = [[element / divider if divider != 0 else 0 for element, divider in zip(row, result_divider_3_2_total)] for row in have_2_nth_capture_total]
print(f'first_RU_have_2_STA_simu = {first_RU_have_2_STA}')
print(f'collision_STA_have_0_nth_simu = {collision_STA_have_0_nth}')
print(f'collision_STA_have_1_nth_simu = {collision_STA_have_1_nth}')
print(f'collision_STA_have_2_nth_simu = {collision_STA_have_2_nth}')
print(f'have_1_nth_capture_simu = {have_1_nth_capture}')
print(f'have_2_nth_capture_simu = {have_2_nth_capture}')
#     for a in collision_STA_have_1_nth_count:
#         for b in a:
#             if b > 1:
#                 print(a)
#     for a in collision_STA_have_2_nth_count:
#         for b in a:
#             if b > 1:
#                 print(a)

first_RU_have_2_STA_simu = [0.001, 0.011, 0.038, 0.116, 0.085, 0.069, 0.114, 0.144, 0.23, 0.271, 0.255, 0.102, 0.016, 0.001, 0.0]
collision_STA_have_0_nth_simu = [[0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0], [1.0, 0.0, 0.08695652173913043, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0], [1.0, 1.0, 0.45484949832775917, 0.028201219512195123, 0.12677878395860284, 0.33883058470764615, 0.7993762993762994, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0], [1.0, 1.0, 1.0, 0.6859756097560976, 0.46959896507115134, 0.2818590704647676, 0.16632016632016633, 0.26423357664233577, 0.44461538461538463, 0.6658415841584159, 0.908875177978168, 1.0, 1.0, 1.0, 0], [1.0, 1.0, 1.0, 1.0, 0.9029754204398448, 0.7781109445277361, 0.46465696465696466, 0.245985401459854, 0.11897435897435897, 0.03712871287128713, 0.0028476506881822496, 0.0, 0.0, 0.0, 0]]
collision_STA_have_1_nth_simu = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0], [0.0, 0.0, 0.458193979933

In [222]:
print(f'have_1_nth_capture_total n=1 = {have_1_nth_capture_total[0]}')
print(f'have_1_nth_capture_total n=2 = {have_1_nth_capture_total[1]}')
print(f'have_1_nth_capture_total n=3 = {have_1_nth_capture_total[2]}')
print(f'have_1_nth_capture_total n=4 = {have_1_nth_capture_total[3]}')
print(f'have_1_nth_capture_total n=5 = {have_1_nth_capture_total[4]}')

print('\n')

print(f'have_2_nth_capture_total n=1 = {have_2_nth_capture_total[0]}')
print(f'have_2_nth_capture_total n=2 = {have_2_nth_capture_total[1]}')
print(f'have_2_nth_capture_total n=3 = {have_2_nth_capture_total[2]}')
print(f'have_2_nth_capture_total n=4 = {have_2_nth_capture_total[3]}')
print(f'have_2_nth_capture_total n=5 = {have_2_nth_capture_total[4]}')


have_1_nth_capture_total n=1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_1_nth_capture_total n=2 = [0, 0, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_1_nth_capture_total n=3 = [0, 0, 76, 131, 110, 95, 67, 0, 0, 0, 0, 0, 0, 0, 0]
have_1_nth_capture_total n=4 = [0, 0, 0, 201, 182, 187, 203, 295, 355, 321, 78, 0, 0, 0, 0]
have_1_nth_capture_total n=5 = [0, 0, 0, 0, 43, 74, 202, 282, 342, 294, 79, 0, 0, 0, 0]


have_2_nth_capture_total n=1 = [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_2_nth_capture_total n=2 = [0, 148, 112, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_2_nth_capture_total n=3 = [0, 0, 23, 761, 264, 91, 12, 0, 0, 0, 0, 0, 0, 0, 0]
have_2_nth_capture_total n=4 = [0, 0, 0, 34, 58, 127, 274, 291, 196, 72, 6, 0, 0, 0, 0]
have_2_nth_capture_total n=5 = [0, 0, 0, 0, 1, 8, 67, 300, 730, 1336, 1573, 750, 141, 4, 0]


In [214]:
print(f'result_divider_3_1_total n=1 = {result_divider_3_1_total[0]}')
print(f'result_divider_3_1_total n=2 = {result_divider_3_1_total[1]}')
print(f'result_divider_3_1_total n=3 = {result_divider_3_1_total[2]}')
print(f'result_divider_3_1_total n=4 = {result_divider_3_1_total[3]}')
print(f'result_divider_3_1_total n=5 = {result_divider_3_1_total[4]}')

print('\n')

print(f'result_divider_3_2_total n=1 = {result_divider_3_2_total[0]}')
print(f'result_divider_3_2_total n=2 = {result_divider_3_2_total[1]}')
print(f'result_divider_3_2_total n=3 = {result_divider_3_2_total[2]}')
print(f'result_divider_3_2_total n=4 = {result_divider_3_2_total[3]}')
print(f'result_divider_3_2_total n=5 = {result_divider_3_2_total[4]}')


result_divider_3_1_total n=1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
result_divider_3_1_total n=2 = [0, 0, 1392, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
result_divider_3_1_total n=3 = [0, 0, 1392, 3980, 3566, 3356, 2018, 0, 0, 0, 0, 0, 0, 0, 0]
result_divider_3_1_total n=4 = [0, 0, 0, 3980, 3364, 3455, 4911, 6799, 8146, 7155, 1916, 0, 0, 0, 0]
result_divider_3_1_total n=5 = [0, 0, 0, 0, 594, 1505, 4195, 6799, 8146, 7155, 1916, 0, 0, 0, 0]


result_divider_3_2_total n=1 = [61, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
result_divider_3_2_total n=2 = [0, 1840, 1322, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
result_divider_3_2_total n=3 = [0, 0, 320, 9367, 3255, 1126, 140, 0, 0, 0, 0, 0, 0, 0, 0]
result_divider_3_2_total n=4 = [0, 0, 0, 348, 822, 1621, 3347, 3343, 2129, 760, 49, 0, 0, 0, 0]
result_divider_3_2_total n=5 = [0, 0, 0, 0, 9, 105, 912, 3315, 8157, 16421, 18537, 8971, 1750, 97, 2]


In [215]:
print(f'have_1_nth_capture n=1 = {have_1_nth_capture[0]}')
print(f'have_1_nth_capture n=2 = {have_1_nth_capture[1]}')
print(f'have_1_nth_capture n=3 = {have_1_nth_capture[2]}')
print(f'have_1_nth_capture n=4 = {have_1_nth_capture[3]}')
print(f'have_1_nth_capture n=5 = {have_1_nth_capture[4]}')

print('\n')


print(f'have_2_nth_capture n=1 = {have_2_nth_capture[0]}')
print(f'have_2_nth_capture n=2 = {have_2_nth_capture[1]}')
print(f'have_2_nth_capture n=3 = {have_2_nth_capture[2]}')
print(f'have_2_nth_capture n=4 = {have_2_nth_capture[3]}')
print(f'have_2_nth_capture n=5 = {have_2_nth_capture[4]}')


have_1_nth_capture n=1 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_1_nth_capture n=2 = [0, 0, 0.3024425287356322, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_1_nth_capture n=3 = [0, 0, 0.5739942528735632, 0.2974874371859296, 0.30818844643858667, 0.29499404052443384, 0.298810703666997, 0, 0, 0, 0, 0, 0, 0, 0]
have_1_nth_capture n=4 = [0, 0, 0, 0.5836683417085428, 0.5713436385255648, 0.557452966714906, 0.4614131541437589, 0.4277099573466686, 0.4300270071200589, 0.4230607966457023, 0.41805845511482254, 0, 0, 0, 0]
have_1_nth_capture n=5 = [0, 0, 0, 0, 0.5202020202020202, 0.46445182724252493, 0.4374255065554231, 0.41506103838799824, 0.40842131107291924, 0.4181691125087352, 0.4164926931106472, 0, 0, 0, 0]


have_2_nth_capture n=1 = [0.7868852459016393, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_2_nth_capture n=2 = [0, 0.8418478260869565, 0.8517397881996974, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
have_2_nth_capture n=3 = [0, 0, 0.871875, 0.8866232518415714, 0.871889400921659, 0.877442

In [216]:
sum_collision_STA = [[sum(element) for element in zip(row0, row1, row2)]
                     for row0, row1, row2 in zip(collision_STA_have_0_nth, collision_STA_have_1_nth, collision_STA_have_2_nth)]


print(f'sum_collision_STA n=1: {sum_collision_STA[0]}')
print(f'sum_collision_STA n=2: {sum_collision_STA[1]}')
print(f'sum_collision_STA n=3: {sum_collision_STA[2]}')
print(f'sum_collision_STA n=4: {sum_collision_STA[3]}')
print(f'sum_collision_STA n=5: {sum_collision_STA[4]}')

sum_collision_STA n=1: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
sum_collision_STA n=2: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
sum_collision_STA n=3: [1.0, 1.0, 0.9999999999999999, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
sum_collision_STA n=4: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
sum_collision_STA n=5: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
